In [2]:
# Setup
# -*- coding: utf-8 -*-
import numpy as np
import html5lib
import scipy as sp
import pandas as p
from IPython.display import display
from sklearn import datasets, linear_model, preprocessing
from sklearn.metrics import mean_squared_error, r2_score
import urllib2
from bs4 import BeautifulSoup 
from bs4 import Comment

In [3]:
# Get salaries for each year and store in salaries_by_year

In [9]:
# teams = 
import requests
teams = ['ARI','ATL','BAL','BOS','CHC','CHW','CIN','CLE','COL','DET',
         'HOU','KC','LAA','LAD','MIA','MIL','MIN','NYM','NYY','OAK','PHI',
         'PIT','SD','SEA','SF','STL','TB','TEX','TOR','WSN']
years = ['2011', '2012', '2013', '2014', '2015', '2016']

stats_train_x = p.DataFrame()
stats_test_x = p.DataFrame()

for year in years:
    # Salary data for a year for ALL players -->
    salaries_df = p.read_csv(year + 'Salary.csv', 
                             engine='python')
    salaries_df['Name'] = salaries_df.Name.str.replace("\\" , '#')
    salaries_df['Name'] = salaries_df.Name.str.replace("#'a0" , ' ')
    salaries_df['Name'] = salaries_df['Name'].str.split('#').str[0]
    salaries_df['Pos'] = salaries_df['Pos'].str[:-1]
    salaries_df['Salary'] = salaries_df['Salary'].str.replace('$','')
    
    # Get batting data for a team -->
    for team in teams:
        url = 'https://www.baseball-reference.com/teams/'+team.upper()+'/'+year+'.shtml'
        batting_df = p.read_html(url)[0]
        # Take columns whose names don't contain "Unnamed"
        batting_df.drop([x for x in batting_df.columns if 'Unnamed' in x], axis=1, inplace=True)
        
        # Filter team's roster to be only those who have a salary
        team_salary_df = salaries_df[salaries_df.Tm == team.upper()]
        team_salary_df = team_salary_df.sort_values('Name')
        
        batting_df = batting_df[batting_df.Name.isin(team_salary_df.Name)]
        team_salary_df = team_salary_df[team_salary_df.Name.isin(batting_df.Name)]
        batting_df = batting_df.sort_values('Name')      
        
        if team == 'ARI':
            train_x = batting_df
            train_y = team_salary_df
        else:
            train_x = p.concat([train_x, batting_df])
            train_y = p.concat([train_y, team_salary_df]) 
    
train_x.drop(train_x.columns[[0, 1, 2, 3, -1]], axis=1, inplace=True)    
train_y = train_y.filter(['Salary'], axis=1)
train_x = train_x.fillna(0)
train_y = train_y.fillna(0)
display(train_x)
display(train_y)


,G,PA,AB,R,H,2B,3B,HR,RBI,SB,...,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF
18,12,46,41,9,10,0,0,2,4,4,...,.244,.326,.390,.716,85,16,1,0,0,0
5,134,499,461,59,130,31,1,16,53,1,...,.282,.329,.458,.786,101,211,14,3,0,4
13,56,166,148,21,42,5,4,6,28,4,...,.284,.352,.493,.845,116,73,2,0,1,1
9,119,466,437,52,121,24,11,5,49,21,...,.277,.315,.416,.731,88,182,8,5,2,2
46,65,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,48,183,171,23,43,9,5,4,15,2,...,.251,.295,.433,.728,86,74,3,3,0,1
38,41,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44,65,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,151,594,523,81,130,31,9,29,91,6,...,.249,.332,.509,.840,114,266,13,3,0,4
2,153,694,637,102,203,41,7,20,64,33,...,.319,.368,.499,.867,122,318,6,12,4,2


,Salary
294,3500000
772,508900
660,516900
553,530500
340,2700000
559,529600
733,511500
787,507500
618,520300
345,2600000


In [6]:
import requests
teams = ['BOS']
years = ['2017']

train_x = p.DataFrame()
train_y = p.DataFrame()

for year in years:
    # Salary data for a year for ALL players -->
    salaries_df = p.read_csv(year + 'Salary.csv', 
                             engine='python')
    salaries_df['Name'] = salaries_df.Name.str.replace("\\" , '#')
    salaries_df['Name'] = salaries_df.Name.str.replace("#'a0" , ' ')
    salaries_df['Name'] = salaries_df['Name'].str.split('#').str[0]
    salaries_df['Pos'] = salaries_df['Pos'].str[:-1]
    salaries_df['Salary'] = salaries_df['Salary'].str.replace('$','')
    
    # Get batting data for a team -->
    for team in teams:
        url = 'https://www.baseball-reference.com/teams/'+team.upper()+'/'+year+'.shtml'
        batting_df = p.read_html(url)[0]
        # Take columns whose names don't contain "Unnamed"
        batting_df.drop([x for x in batting_df.columns if 'Unnamed' in x], axis=1, inplace=True)
        # Filter team's roster to be only those who have a salary
        team_salary_df = salaries_df[salaries_df.Tm == team.upper()]   
        batting_df = batting_df[batting_df.Name.isin(team_salary_df.Name)]
        team_batting_salary_df = team_salary_df[team_salary_df.Name.isin(batting_df.Name)]
        batting_df = batting_df.sort_values('Name')      
        display(batting_df)
        
        # Pitching df
        pitching_df = p.read_html(url)[1]
        pitching_df.drop([x for x in pitching_df.columns if 'Unnamed' in x], axis=1, inplace=True)
        pitching_df = pitching_df[pitching_df.Name.isin(team_salary_df.Name)]
        team_salary_df = team_salary_df[team_salary_df.Name.isin(pitching_df.Name)]
        pitching_df = pitching_df.sort_values('Name')           
        
        display(pitching_df)
    
# test_x.drop(test_x.columns[[0, 1, 2, 3, -10, -1]], axis=1, inplace=True)    
# test_y = test_y.filter(['Salary'], axis=1)
# test_x = test_x.fillna(0)
# test_y = test_y.fillna(0)


,Rk,Pos,Name,Age,G,PA,AB,R,H,2B,...,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB
5,6,LF,Andrew Benintendi*,22,151,658,573,84,155,26,...,.352,.424,.776,103,243,16,6,1,8,7
38,37,P,Ben Taylor,24,14,0,0,0,0,0,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
44,43,P,Brandon Workman,28,33,0,0,0,0,0,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
14,14,2B,Brock Holt*,29,64,164,140,20,28,6,...,.305,.243,.548,47,34,3,3,0,2,0
34,33,P,Carson Smith,27,8,0,0,0,0,0,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
27,26,P,Chris Sale*,28,32,3,3,0,1,1,...,.333,.667,1.000,152,2,0,0,0,0,0
11,11,UT,Chris Young,33,90,276,243,30,57,12,...,.322,.387,.709,86,94,4,2,0,1,0
0,1,C,Christian Vazquez,26,99,345,324,43,94,18,...,.330,.404,.735,92,131,14,3,0,1,0
39,38,P,Craig Kimbrel,29,67,0,0,0,0,0,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
33,32,P,David Price*,31,16,1,1,0,1,0,...,1.000,1.000,2.000,430,1,0,0,0,0,0


,Rk,Pos,Name,Age,W,L,W-L%,ERA,G,GS,...,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W
21,20,NaN,Ben Taylor,24,0,1,.000,5.19,14,0,...,1,80,89,4.89,1.673,10.4,1.6,4.7,9.3,2.00
15,14,NaN,Brandon Workman,28,1,1,.500,3.18,33,0,...,1,162,144,4.49,1.210,8.4,1.6,2.5,8.4,3.36
24,23,NaN,Carson Smith,27,0,0,NaN,1.35,8,0,...,0,27,354,1.96,1.350,9.5,0.0,2.7,9.5,3.50
0,1,SP,Chris Sale*,28,17,8,.680,2.90,32,32,...,3,851,157,2.45,0.970,6.9,1.0,1.8,12.9,7.16
7,7,CL,Craig Kimbrel,29,5,0,1.000,1.43,67,0,...,5,254,319,1.42,0.681,4.3,0.8,1.8,16.4,9.00
5,6,SP,David Price*,31,6,3,.667,3.38,16,11,...,2,317,135,3.64,1.192,7.8,1.0,2.9,9.2,3.17
3,4,SP,Eduardo Rodriguez*,24,6,7,.462,4.19,25,24,...,1,582,109,3.97,1.282,8.3,1.2,3.3,9.8,3.00
13,12,NaN,Fernando Abad*,31,2,1,.667,3.30,48,0,...,0,182,139,3.68,1.237,8.2,0.8,2.9,7.6,2.64
9,9,RP,Heath Hembree,28,2,3,.400,3.63,62,0,...,2,271,126,3.92,1.452,10.5,1.5,2.6,10.2,3.89
10,10,RP,Joe Kelly,29,4,1,.800,2.79,54,0,...,4,238,164,3.49,1.190,6.5,0.5,4.2,8.1,1.93


In [10]:
regr = linear_model.LinearRegression()
print train_x.values
regr.fit(train_x.values, train_y.values)
# salary_predict(testingYear)
print('Weights Bitch!!!! = \n', regr.coef_)
predicted = regr.predict(test_x.values)
# print test_y.values
test_y.Salary = test_y.Salary.astype(float)
diff = predicted - (test_y.values)
binarizer = preprocessing.Binarizer().fit(diff)
binarizer.transform(diff)

# overpaid = np.where((predicted - (test_y.values)) < -10000000)[0]
# for over in overpaid: 
#     display(test_x.iloc[over])
    


[[u'12' u'46' u'41' ..., u'0' u'0' u'0']
 [u'134' u'499' u'461' ..., u'3' u'0' u'4']
 [u'56' u'166' u'148' ..., u'0' u'1' u'1']
 ..., 
 [u'33' u'75' u'64' ..., u'0' u'6' u'0']
 [u'131' u'523' u'482' ..., u'2' u'0' u'4']
 [u'33' u'7' u'6' ..., u'0' u'1' u'0']]
('Weights Bitch!!!! = \n', array([[   -53794.22674246,   1015510.68694293,   -997123.18425889,
           -48590.39483554,     97568.68912792,   -184947.14020542,
          -479598.86154744,    327842.53250808,     46982.642674  ,
            65920.57419582,   -386456.34009846,   -878235.66444479,
           -59947.109395  ,  13991524.01135149,    193074.90336235,
          8714970.07727876,  -3293552.44091727,    -32879.56112309,
           -63048.57664921,    140730.39010144,   -959500.60943196,
          -759518.21860031,  -1084096.92192828]]))


ValueError: shapes (590,22) and (23,1) not aligned: 22 (dim 1) != 23 (dim 0)